In [2]:
# 구글 플레이스토어 크롤링 lib 
!pip install google-play-scraper

# 애플 앱스토어 크롤링 lib
!pip install app-store-scraper


[notice] A new release of pip is available: 25.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 25.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [11]:
#######################################
##### 1. 플레이스토어 리뷰 크롤링 #####
#######################################

from google_play_scraper import reviews, Sort
import pandas as pd
import datetime
import time

# 라프텔 앱 ID
app_id = 'laftel.net.laftel'

# 최대 수집 리뷰 수: 3천 개로 제한
max_reviews = 3000
all_reviews = []
next_token = None

# 크롤링 시작
while len(all_reviews) < max_reviews:
    new_reviews, next_token = reviews(
        app_id,
        lang='ko',         # 한국어 리뷰만
        country='kr',      # 한국
        sort=Sort.NEWEST,  # 최신순 정렬
        count=100,         # 100개씩 스크랩핑
        continuation_token=next_token
    )
    all_reviews.extend(new_reviews)
    # print(f"수집된 리뷰 수: {len(all_reviews)}")  # for debug
    
    if not next_token:
        break
    time.sleep(1)  # 서버 과부하 방지

# DataFrame으로 변환
origin_playstore_df = pd.DataFrame(all_reviews)

# 개수 확인
print("✅ 크롤링 완료")
print(f"수집된 플레이스토어 리뷰 수: {len(origin_playstore_df)}")

✅ 크롤링 완료
수집된 플레이스토어 리뷰 수: 3000


In [12]:
print(origin_playstore_df.columns)  # 리뷰 데이터 열 이름 확인

Index(['reviewId', 'userName', 'userImage', 'content', 'score',
       'thumbsUpCount', 'reviewCreatedVersion', 'at', 'replyContent',
       'repliedAt', 'appVersion'],
      dtype='object')


In [15]:
origin_playstore_df.head()

,reviewId,userName,userImage,content,score,thumbsUpCount,reviewCreatedVersion,at,replyContent,repliedAt,appVersion
0,ae017c02-4fb4-4b2d-b39d-6eb6a18aae17,김현지,https://play-lh.googleusercontent.com/a/ACg8oc...,"라프텔 어플을 핸드폰이랑 갤럭시탭에 다 설치해놓고 쓰고있는데, 지지난주부터 갤럭시탭...",2,0,3.67.0,2025-07-20 14:49:36,None,NaT,3.67.0
1,586743e7-3577-4683-90af-7e11592c60f5,정호영,https://play-lh.googleusercontent.com/a/ACg8oc...,굿,5,0,None,2025-07-20 12:14:45,None,NaT,None
2,a58095c6-c5e6-4357-84f6-f0c933a0bde6,김규연,https://play-lh.googleusercontent.com/a/ACg8oc...,아아 왜 무료보기가 없어졌어요-!! 물론 청소년이라서 보려면 부모님한테 허락 받아야...,1,0,None,2025-07-20 10:31:27,None,NaT,None
3,a279225d-57ef-4d1c-bac7-5428ab5a24af,김윤서,https://play-lh.googleusercontent.com/a-/ALV-U...,굿,5,0,3.67.0,2025-07-19 23:36:27,None,NaT,3.67.0
4,daba8593-0122-41be-95cf-4e9336a13d59,태극기휘바이든,https://play-lh.googleusercontent.com/a-/ALV-U...,마도조사. 최고.,5,0,None,2025-07-19 21:35:37,None,NaT,None


In [31]:
#######################################
####### 2. 앱스토어 리뷰 크롤링 #######
#######################################

from app_store_scraper import AppStore
from pprint import pprint

# 앱 리뷰 가져오기
app = AppStore(country="kr", app_name="laftel", app_id=1169440095)  # 국가와 앱 이름 지정

print(app)

app.review(how_many=1000)  # 일단 1천 개 시도

# 리뷰 데이터를 DataFrame으로 변환
review_list = app.reviews
origin_appstore_df = pd.DataFrame(review_list)

pprint(review_list)
print(origin_appstore_df.columns)
print(origin_appstore_df.head(1).to_dict())

# # 최신순 정렬
# origin_appstore_df = origin_appstore_df.sort_values('date', ascending=False)

# # 개수 확인
# print("✅ 크롤링 완료")
# print(f"수집된 앱스토어 리뷰 수: {len(origin_appstore_df)}")

2025-07-21 20:34:50,393 [INFO] Base - Initialised: AppStore('kr', 'laftel', 1169440095)
2025-07-21 20:34:50,394 [INFO] Base - Ready to fetch reviews from: https://apps.apple.com/kr/app/laftel/id1169440095


     Country | kr
        Name | laftel
          ID | 1169440095
         URL | https://apps.apple.com/kr/app/laftel/id1169440095
Review count | 0


2025-07-21 20:34:50,614 [ERROR] Base - Something went wrong: Expecting value: line 1 column 1 (char 0)
2025-07-21 20:34:50,614 [INFO] Base - [id:1169440095] Fetched 0 reviews (0 fetched in total)


[]
RangeIndex(start=0, stop=0, step=1)
{}


In [ ]:
print(origin_appstore_df.columns)  # 리뷰 데이터 열 이름 확인

In [ ]:
origin_appstore_df.head()

In [ ]:
##### 앱스토어 리뷰 수집 불가 이슈로 DB 통합 중단..